## アフターコーディングに向けて実践

In [2]:
pip install pyknp

     |████████████████████████████████| 43 kB 425 kB/s eta 0:00:01
  Created wheel for pyknp: filename=pyknp-0.4.6-py3-none-any.whl size=40482 sha256=39fabf7de682b154ec3fb1fc5894c4c37df1ab4bec41faf5b4e42e40a8e30381
  Stored in directory: /Users/yuto/Library/Caches/pip/wheels/0d/4a/72/89b321939c9889a8557cf087a03a7290b6f4dd405377e6e3c7
Successfully built pyknp
Note: you may need to restart the kernel to use updated packages.


In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import word2vec
import MeCab
import mojimoji
import numpy as np
import pandas as pd
from pyknp import Juman
#次元圧縮によって計算量を減らすために主成分分析をかけるため
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
def juman_text_to_word_genkei(text, stopword_pass="./stopwords/Japanese.txt"):
    """JUMAN++を使い、名詞・動詞・形容詞の原型をリストに格納する関数"""
    #stopwordのリスト作成
    stopword_list =[]
    with open(stopword_pass, "r") as file:
        lines = file.readlines()
            
        stopword_list = [stopword for stopword in lines if stopword.strip()]
        
    #mojimojiで全角数字、英字を半角に統一。大文字もすべて小文字に統一する
    try:
        text = mojimoji.zen_to_han(text, kana=False).lower()
    except:
        pass
    
    jumanpp = Juman()
    #形態素解析する
    result = jumanpp.analysis(text)
    #分類された各形態素ごとに見ていく
    morph_list = result.mrph_list()
    
    basic_words =[]
    for morph in morph_list:
        #記号なら何もせず続ける
        if morph.bunrui == "記号":
            continue
        elif morph.hinsi =="名詞" and morph.genkei not in stopword_list:
            basic_words.append(morph.genkei)
        elif morph.hinsi in ["動詞", "形容詞"] and morph.genkei not in stopword_list:
            basic_words.append(morph.genkei)
            
    #basic_wordsのリストを半角スペースでまとめておく
    basic_words = " ".join(basic_words)
    return basic_words

In [5]:
def juman_text_to_word_wakati(text, stopword_pass="./stopwords/Japanese.txt"):
    """JUMAN++を使い、名詞・動詞・形容詞の原型をリストに格納する関数"""
    #stopwordのリスト作成
    stopword_list =[]
    with open(stopword_pass, "r") as file:
        lines = file.readlines()
            
        stopword_list = [stopword for stopword in lines if stopword.strip()]
        
    #mojimojiで全角数字、英字を半角に統一。大文字もすべて小文字に統一する
    #0305修正
    text = mojimoji.zen_to_han(text, kana=False).lower()

    jumanpp = Juman()
    #形態素解析する
    result = jumanpp.analysis(text)
    #分類された各形態素ごとに見ていく
    morph_list = result.mrph_list()
    
    basic_words =[]
    for morph in morph_list:
        basic_words.append(morph.midasi)
            
    #basic_wordsのリストを半角スペースでまとめておく
    basic_words = " ".join(basic_words)
    return basic_words

In [12]:
test_txt = pd.read_csv("【SCR1万】「具たっぷり味噌汁」ベンチマーク調査2020_RawData_AC作業_途中確認用3.txt",header=0,encoding="cp932",sep="\t")
test_txt.head()

,RMID,SC10t1,SC10t2,SC10t3,SC10t4,SC10t5,NSC10_1,NSC10_2c1,NSC10_2c2,NSC10_2c3,...,NSC10_2c8,NSC10_2c9,NSC10_2c10,NSC10_2c11,NSC10_2c12,NSC10_2c13,NSC10_2c14,NSC10_2c15,NSC10_2c16,NSC10_2c17
0,1000000075,ない,NaN,NaN,NaN,NaN,17,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1000000086,なし,NaN,NaN,NaN,NaN,17,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1000000217,あさげ,アマノフーズ,NaN,NaN,NaN,11,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1000000226,あさげ,ひるげ,NaN,NaN,NaN,11,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,1000000236,あさげ,NaN,NaN,NaN,NaN,11,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
test_txt_adjust = test_txt[["RMID", "SC10t1", "SC10t2", "SC10t3", "SC10t", "SC10t1"]]

In [14]:
#一旦絵文字が出てきた場合は空白に置換
import re
test_txt.replace("&#[\d]{4,6}"," ",regex=True,inplace=True)

In [15]:
#整形
test_txt.replace("\n","",inplace=True,regex=True)
test_txt.replace("\d","",inplace=True,regex=True)

In [16]:
#口コミの列を形態素解析していく
#一旦Q2FAで検証
#test_txt["SC10t1_genkei"] = test_txt["SC10t1"][:1000].map(juman_text_to_word_genkei)
test_txt["SC10t1_wakati"] = test_txt["SC10t1"][:3000].map(juman_text_to_word_wakati)

In [ ]:
sample_corpus = test_txt["SC10t1_wakati"]
# 出現する単語のカウントを特徴量にする手法になります。
# 出現した単語を純粋にカウントします。
vec = CountVectorizer()
#fit_transformで文書行列を取得
vec_fit_transform = vec.fit_transform(sample_corpus)

print('Bag of Words')
#カラムに全ての単語の種類を提示、インデックスは各文章を提示
display(pd.DataFrame(vec_fit_transform.toarray(), columns=vec.get_feature_names(), index=sample_corpus))

print('tfidf')
# skleanの中のTfidfTransformerでインスタンス化して利用する
tfidf = TfidfTransformer()

model_tfidf = tfidf.fit_transform(vec_fit_transform)

display(pd.DataFrame(vec_fit_transform.toarray(), columns=vec.get_feature_names(), index=sample_corpus))

In [17]:
test_txt.head(30)

,RMID,SC10t1,SC10t2,SC10t3,SC10t4,SC10t5,NSC10_1,NSC10_2c1,NSC10_2c2,NSC10_2c3,...,NSC10_2c9,NSC10_2c10,NSC10_2c11,NSC10_2c12,NSC10_2c13,NSC10_2c14,NSC10_2c15,NSC10_2c16,NSC10_2c17,SC10t1_wakati
0,1000000075,ない,NaN,NaN,NaN,NaN,17,0,0,0,...,0,0,0,0,0,0,0,0,1,ない
1,1000000086,なし,NaN,NaN,NaN,NaN,17,0,0,0,...,0,0,0,0,0,0,0,0,1,なし
2,1000000217,あさげ,アマノフーズ,NaN,NaN,NaN,11,0,0,0,...,0,0,1,0,0,0,0,0,0,あさげ
3,1000000226,あさげ,ひるげ,NaN,NaN,NaN,11,0,0,0,...,0,0,1,1,0,0,0,0,0,あさげ
4,1000000236,あさげ,NaN,NaN,NaN,NaN,11,0,0,0,...,0,0,1,0,0,0,0,0,0,あさげ
5,1000000237,永谷園,アマノ,NaN,NaN,NaN,2,0,1,0,...,0,0,0,0,0,0,0,0,0,永谷 園
6,1000000238,あさげ,ゆうげ,NaN,NaN,NaN,11,0,0,0,...,0,0,1,0,1,0,0,0,0,あさげ
7,1000000338,はなまるき,ひかり味噌,たけや,NaN,NaN,6,0,0,0,...,0,0,0,0,0,0,0,1,0,はな まるき
8,1000000406,永谷園,NaN,NaN,NaN,NaN,2,0,1,0,...,0,0,0,0,0,0,0,0,0,永谷 園
9,1000000410,なし,NaN,NaN,NaN,NaN,17,0,0,0,...,0,0,0,0,0,0,0,0,1,なし
